In [6]:
# importamos las cosas que vamos a usar de aima
from search import *
import itertools
class ProblemaAscensores(Problem):
    def __init__(self, initial, goal):
        self.goal = goal
        self.initial = initial
        self.analizados = 0
        Problem.__init__(self, initial, goal)
        #initial = ( (pisos de las personas),(0el piso que situa ascensor, 1range inicio de ascensor, 2range final de ascensor,
        #            3booleano de que si lleva persona, 4posicion de persona en la lista de personas, 5destino piso a llegar),... los demas ascensores )
        
    def goal_test(self,estado): 
        self.analizados += 1
        if (estado[0] == self.goal):
            j = False
            for i in estado[1:]:
                if i[3]==1:
                    j = j or True
            if j == False:
                return True
        return False
    
    def listaDeNoLlegados(self, estado):
        # lista de las personas que no han llegado a su destino
        lista_final = list()
        lista_de_nollegados = list()
        lista_de_pers_en_asc = list()
        lista_de_pos_nollegados = list()
        for i in range(len(estado[1:])):
            if estado[i+1][3] == 1:
                lista_de_pers_en_asc.append(estado[i+1][4])        
        for i in range(len(estado[0])):
            if estado[0][i] != self.goal[i] and i not in lista_de_pers_en_asc:
                lista_de_pos_nollegados.append(i)
        for i in lista_de_pos_nollegados:
            lista_de_nollegados.append(estado[0][i])
        lista_final.append(lista_de_nollegados)
        lista_final.append(lista_de_pos_nollegados)
        return lista_final
    
    def actions(self, estado):
        acs = list()
        accs = list()
        
        lista_de_nollegados = self.listaDeNoLlegados(estado)
        for asc in estado[1:]:
            accaux = list()
            if asc[3] == 0:
                if asc[0] in lista_de_nollegados[0]:
                    destino = self.goal[lista_de_nollegados[1][lista_de_nollegados[0].index(asc[0])]]
                if asc[0] in lista_de_nollegados[0] and ((asc[0] < asc[2] and asc[0] < destino) or (asc[0] > asc[1] and asc[0] > destino)):
                    accaux.append("entrar")
                else:
                    pos_por_encima = list(filter(lambda x: (x >asc[0] and x <= asc[2]), lista_de_nollegados[0]))
                    pos_por_abajo = list(filter(lambda x: (x <asc[0] and x >= asc[1]), lista_de_nollegados[0]))
                    if len(pos_por_encima)>0:
                        accaux.append("subir")
                    if len(pos_por_abajo)>0:
                        accaux.append("bajar")
                    if len(pos_por_encima) == 0 and len(pos_por_abajo) == 0:
                        accaux.append("nada")
            elif asc[3] == 1:
                if asc[0] == asc[5] or (asc[5]> asc[2] and asc[0] == asc[2]) or (asc[5] < asc[1] and asc[0] == asc[1]):
                    accaux.append("salir")
                 
                elif asc[0] < asc[5] and asc[0] < asc[2]:
                    accaux.append("subir")
                elif asc[0] > asc[5] and asc[0] > asc[1]:
                    accaux.append("bajar")
            accs.append(accaux)
            del accaux
        for i in itertools.product(*accs):
            acs.append(i)
        return acs
    
    def result(self, estado, a):
        tup = list()
        list_estado = list()
        for i in estado:
            list_estado.append(list(i))
        idc = 1
        for acc in a:
            asc = list(estado[idc])
            if acc == "entrar":
                lista_de_nollegados = self.listaDeNoLlegados(list_estado)
                if asc[0] in lista_de_nollegados[0]:
                    pers = lista_de_nollegados[0].index(asc[0])
                    pos = lista_de_nollegados[1][pers]
                    asc[3] = 1
                    asc[4] = pos
                    asc[5] = self.goal[pos]
            elif acc == "salir":
                asc[3] = 0
            elif acc == "subir":
                asc[0] += 1
                if asc[3] == 1:
                    pos_pers = asc[4]
                    list_estado[0][pos_pers] = asc[0]
            elif acc == "bajar":
                asc[0] -= 1
                if asc[3] == 1:
                    pos_pers = asc[4]
                    list_estado[0][pos_pers] = asc[0]
            list_estado[idc] = asc
            del asc
            idc += 1
        for i in list_estado:
            aux = tuple(i)
            tup.append(aux)
        return tuple(tup)
                            
    def h1(node):
        h = 0
        estado = node.state
        lista = node.listaDeNoLlegados(estado)  
        for i in range(len(estado[0])):
            h += abs(estado[0][i] - goal[i])
        

In [8]:
ascensores = ProblemaAscensores(((2,6,9),(0,0,7,0,0,0),(9,7,12,0,0,0)),(0,12,7)) 
breadth_first_graph_search(ascensores).solution()

[('subir', 'entrar'),
 ('subir', 'bajar'),
 ('entrar', 'bajar'),
 ('bajar', 'salir'),
 ('bajar', 'nada'),
 ('salir', 'nada'),
 ('subir', 'nada'),
 ('subir', 'nada'),
 ('subir', 'nada'),
 ('subir', 'nada'),
 ('subir', 'nada'),
 ('subir', 'nada'),
 ('entrar', 'nada'),
 ('subir', 'nada'),
 ('salir', 'nada'),
 ('nada', 'entrar'),
 ('nada', 'subir'),
 ('nada', 'subir'),
 ('nada', 'subir'),
 ('nada', 'subir'),
 ('nada', 'subir'),
 ('nada', 'salir')]

In [9]:
ascensores = ProblemaAscensores(((2,7,9),(7,0,7,1,1,12),(9,7,12,0,0,0)),(0,12,7)) 
ascensores.actions(ascensores.initial)

[('salir', 'entrar')]

In [10]:
ascensores.result(ascensores.initial, ('subir', 'bajar'))

((2, 8, 9), (8, 0, 7, 1, 1, 12), (8, 7, 12, 0, 0, 0))

In [11]:
%%time
problema = ProblemaAscensores(((2,4,1,8,1),(0,0,4,0,0,0),(5,4,8,0,0,0),(5,4,8,0,0,0),(9,8,12,0,0,0)),(3,11,12,1,9))

Wall time: 0 ns


In [13]:
%%time
depth_first_graph_search(problema).solution()

Wall time: 0 ns


[('subir', 'bajar', 'bajar', 'bajar'),
 ('entrar', 'entrar', 'entrar', 'nada'),
 ('subir', 'subir', 'subir', 'nada'),
 ('subir', 'subir', 'subir', 'nada'),
 ('subir', 'subir', 'subir', 'nada'),
 ('salir', 'subir', 'subir', 'nada'),
 ('bajar', 'salir', 'entrar', 'nada'),
 ('bajar', 'entrar', 'salir', 'nada'),
 ('entrar', 'bajar', 'bajar', 'entrar'),
 ('subir', 'bajar', 'bajar', 'subir'),
 ('salir', 'bajar', 'bajar', 'subir'),
 ('bajar', 'bajar', 'bajar', 'subir'),
 ('bajar', 'salir', 'entrar', 'salir'),
 ('entrar', 'nada', 'subir', 'nada'),
 ('subir', 'nada', 'subir', 'nada'),
 ('subir', 'nada', 'subir', 'nada'),
 ('subir', 'nada', 'subir', 'nada'),
 ('salir', 'nada', 'salir', 'nada'),
 ('entrar', 'subir', 'bajar', 'bajar'),
 ('bajar', 'bajar', 'bajar', 'bajar'),
 ('bajar', 'entrar', 'bajar', 'bajar'),
 ('bajar', 'subir', 'subir', 'entrar'),
 ('salir', 'subir', 'nada', 'subir'),
 ('nada', 'subir', 'nada', 'subir'),
 ('nada', 'subir', 'nada', 'subir'),
 ('nada', 'salir', 'nada', 'subir')

In [15]:
%%time
uniform_cost_search(problema).solution()

Wall time: 18.8 s


[('subir', 'bajar', 'subir', 'bajar'),
 ('entrar', 'entrar', 'subir', 'nada'),
 ('subir', 'subir', 'subir', 'nada'),
 ('subir', 'subir', 'entrar', 'nada'),
 ('subir', 'subir', 'bajar', 'nada'),
 ('salir', 'subir', 'bajar', 'nada'),
 ('bajar', 'salir', 'bajar', 'nada'),
 ('bajar', 'bajar', 'bajar', 'entrar'),
 ('entrar', 'bajar', 'salir', 'subir'),
 ('subir', 'bajar', 'entrar', 'subir'),
 ('salir', 'bajar', 'subir', 'subir'),
 ('bajar', 'nada', 'subir', 'salir'),
 ('bajar', 'nada', 'subir', 'nada'),
 ('entrar', 'nada', 'subir', 'nada'),
 ('subir', 'nada', 'salir', 'nada'),
 ('subir', 'subir', 'bajar', 'bajar'),
 ('subir', 'bajar', 'bajar', 'bajar'),
 ('salir', 'subir', 'bajar', 'bajar'),
 ('entrar', 'subir', 'bajar', 'entrar'),
 ('bajar', 'bajar', 'entrar', 'subir'),
 ('bajar', 'nada', 'subir', 'subir'),
 ('bajar', 'nada', 'subir', 'subir'),
 ('salir', 'nada', 'subir', 'subir'),
 ('nada', 'nada', 'subir', 'salir'),
 ('nada', 'nada', 'salir', 'nada'),
 ('nada', 'subir', 'nada', 'bajar'),

In [14]:
%%time
astar_search(problema, (lambda x: 1)).solution()

Wall time: 23.5 s


[('subir', 'bajar', 'subir', 'bajar'),
 ('entrar', 'entrar', 'subir', 'nada'),
 ('subir', 'subir', 'subir', 'nada'),
 ('subir', 'subir', 'entrar', 'nada'),
 ('subir', 'subir', 'bajar', 'nada'),
 ('salir', 'subir', 'bajar', 'nada'),
 ('bajar', 'salir', 'bajar', 'nada'),
 ('bajar', 'bajar', 'bajar', 'entrar'),
 ('entrar', 'bajar', 'salir', 'subir'),
 ('subir', 'bajar', 'entrar', 'subir'),
 ('salir', 'bajar', 'subir', 'subir'),
 ('bajar', 'nada', 'subir', 'salir'),
 ('bajar', 'nada', 'subir', 'nada'),
 ('entrar', 'nada', 'subir', 'nada'),
 ('subir', 'nada', 'salir', 'nada'),
 ('subir', 'subir', 'bajar', 'bajar'),
 ('subir', 'bajar', 'bajar', 'bajar'),
 ('salir', 'subir', 'bajar', 'bajar'),
 ('entrar', 'subir', 'bajar', 'entrar'),
 ('bajar', 'bajar', 'entrar', 'subir'),
 ('bajar', 'nada', 'subir', 'subir'),
 ('bajar', 'nada', 'subir', 'subir'),
 ('salir', 'nada', 'subir', 'subir'),
 ('nada', 'nada', 'subir', 'salir'),
 ('nada', 'nada', 'salir', 'nada'),
 ('nada', 'subir', 'nada', 'bajar'),